In [15]:
import os
os.environ["LANGCHAIN_TELEMETRY"] = "false"
os.environ["CHROMA_TELEMETRY"] = "0"
os.environ["ANONYMIZED_TELEMETRY"] = "false"


from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.llms import huggingface_pipeline

In [16]:
# ---------------------------
# 1. Load PDF documents
# ---------------------------
docs_dir = "docs"
pdf_files = [os.path.join(docs_dir, f) for f in os.listdir(docs_dir) if f.endswith(".pdf")][:10]

documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pages = loader.load_and_split()
    documents.extend(pages)

print(f"Loaded {len(documents)} pages from {len(pdf_files)} PDFs.")


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)


Loaded 297 pages from 10 PDFs.


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

In [24]:
# ---------------------------
# 2. Create embeddings (Hugging Face)
# ---------------------------
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)


In [26]:
# ---------------------------
# 3. Store embeddings in Chroma
# ---------------------------
from langchain_chroma import Chroma
vector_db_dir = "./chroma_db4"
if not os.path.exists(vector_db_dir):
    os.makedirs(vector_db_dir)

db = Chroma.from_documents(chunks, embeddings, persist_directory=vector_db_dir)
print("Embeddings stored in Chroma vector database.")



Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Embeddings stored in Chroma vector database.


In [27]:
# ---------------------------
# 4. Create LangChain retriever
# ---------------------------
retriever = db.as_retriever()


In [28]:
# ---------------------------
# 5. Create Hugging Face LLM
# ---------------------------
# Text generation pipeline (can use "hkunlp/instructor-large" or similar)
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
# from langchain_huggingface import HuggingFacePipeline
hf_pipeline = pipeline(
    "text-generation",
    model="google/flan-t5-small",
    max_length=512
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FlexOlmoF

In [29]:
# ---------------------------
# 6. Create RetrievalQA chain
# ---------------------------
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)


In [30]:
# ---------------------------
# 7. Ask questions interactively
# ---------------------------
while True:
    query = input("\nEnter your question (or 'exit' to quit): ")
    if query.lower() == "exit":
        break

    answer = qa_chain.invoke({"query": query})
    print("\nAnswer (LangChain RAG):")
    print(answer["result"])


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
Token indices sequence length is longer than the specified maximum sequence length for this model (1179 > 512). Running this sequence through the model will result in indexing errors



Answer (LangChain RAG):
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

101 9 Maschinelles Lernen  zur Bestimmung von  up  
Dieses Kapitel beschreibt, wie es möglich ist, Verfahren des maschinellen Lernens zu nutzen, 
um den Algorithmus 𝑢𝑝 anhand kleine r Menge n gelabelte r Beispiele zu lernen.  
Entsprechend seiner Anforderungen (siehe Abschnitt 7.9) muss 𝑢𝑝 deterministisch die 
unbekannte Variablenabstraktion  𝛼𝑢𝑝 nachbilden. Da 𝛼𝑢𝑝 definitionsgemäß (siehe Abschnitt 
7.4) eine surjektive Abbildung ist, handelt es sich bei der Bestimmung von 𝑢𝑝 um ein 
Regressionsproblem.  
Zur Lösung derartiger  Problem e gibt es eine Reihe von etablierten Verfahren.  Die ausgewählter  
Verfahren zur Bestimmung von 𝑢𝑝 wurde bereits betrachtet  (Wittek, 2018) . Hierzu wurden 
unter Anderem  vier Trainingsmengen mit jeweils 20 Datenpunkten ausgewählt . Als 
Abstraktionsfunktion wurden d